# Julia で連分数

Fresnelの余弦積分，Fresnelの正弦積分をそれぞれ次で定義する:
$$C(x)=\int_{0}^{x} \cos\left(\frac{\pi}{2}t^2\right)dt , \quad S(x)=\int_{0}^{x} \sin\left(\frac{\pi}{2}t^2\right)dt.$$
このとき
$$C(x)+iS(x)=\int_{0}^{x} e^{i\pi t^2/2}dt$$
となる．[2] では $x\geq 2$ の時に次の式を用いて計算している：
$$\int_{0}^{x} e^{i\pi t^2/2}dt=\frac{1+i}{2}-\frac{i}{\pi x} e^{i\pi x^2/2}H(z).$$
ここで$z=i/\pi x^2$ であり $H(z)$ は連分数で定まる次の形をしている関数である．
$$H(z)=\frac{1}{1+\frac{z}{1+\frac{2z}{1+\frac{3z}{1+...}}}}$$

というわけでJuliaでFresnel積分の（スクラッチによる）実装をしてみたかったときに連分数の計算が必要になった.

## 方法１(Reference [1])

In [1]:
using SymPy

In [2]:
function f(z::Number, n::Integer)
    cf::typeof(inv(z))=1
    for k in n:-1:1
        cf = 1+k*z/cf
    end
    return inv(cf)
end

f (generic function with 1 method)

In [3]:
display("text/latex",f(Sym(:z),6))

## 方法２(Reference [2])


\begin{align}
&A_{-1}=1,B_{-1}=0, \\
&A_{0}=1,B_{0}=1, \\
&A_k=A_{k-1}+kzA_{k-2} \quad (k=1,2,3,...)\\
&B_k=B_{k-1}+kzB_{k-2} \quad (k=1,2,3,...)
\end{align}

で定まる数列 $A_N, B_N$ を計算して $1/(A_N/B_N)$ とする．


In [4]:
function g(z::Number, n::Integer)
    a_old=1 ; b_old=0
    a_now=1 ; b_now=1
    a=0
    b=0
    for k in 1:n
        a=a_now+k*z*a_old
        b=b_now+k*z*b_old
        a_old=a_now
        b_old=b_now
        a_now=a
        b_now=b
    end
    print("a=$a b=$b")
    return inv(a/b)
end

g (generic function with 1 method)

In [5]:
display("text/latex",g(Sym(:z),6))

a=3*z*(z + 1) + 4*z*(3*z + 1) + 5*z*(3*z*(z + 1) + 3*z + 1) + 6*z*(3*z*(z + 1) + 4*z*(3*z + 1) + 3*z + 1) + 3*z + 1 b=4*z*(2*z + 1) + 5*z*(5*z + 1) + 6*z*(4*z*(2*z + 1) + 5*z + 1) + 5*z + 1

## 数値での比較

In [6]:
a=rand()
n=100
f(a,n)-g(a,n) <1e-15

a=1.72468885394813e64 b=1.3750788773870185e64

true

In [7]:
@time f(a,n)

  0.000006 seconds (84 allocations: 6.107 KiB)


0.7972909862780239

In [8]:
@time g(a,n)

a=1.72468885394813e64 b=1.3750788773870185e64  0.000238 seconds (1.30 k allocations: 21.016 KiB)


0.7972909862780234

## g について
-  n=1000とかにするとa,bともに発散してどうしようもなくなる
- 上のtime測定でもわかるようにメモリの使用量もちがうっぽい

In [9]:
a=5.5
n=1000
@time g(a,n)

a=Inf b=Inf  0.000308 seconds (13.07 k allocations: 204.641 KiB)


NaN

# Reference:
- [1] :https://twitter.com/genkuroki/status/898008651033489408
- [2] :田口俊弘著 Fortan ハンドブック 270頁 Key Element6.1 連分数の計算方法 技術評論社より
- [3] :http://www.aip.de/groups/soe/local/numres/bookcpdf/c6-9.pdf